# Item 42. compareTo의 규약을 지켜라
- Comparable 인터페이스는 compareTo 메서드를 정의한다. Object의 메서드가 아니다.

## compareTo 규약
- 비대칭적 동작 : a >=b 이고 b >= a 라면 a == b 여야 한다. 즉 비교와 동등성 비교에 어떤 관계가 있어야하며, 서로 일관성이 있어야한다
- 연속적 동작(추이성) : a >= b 이고 b >= c 라면 a >= c 여야 한다. 즉 비교가 연속적이어야 한다.
- 코넥스적 동작 : 두 요소는 어떤 확실한 관계(a>=b 이든 b>=a 이든)가 있어야 한다.



In [6]:
data class NonComparable(val value: Int, val value2: Int)

val x = NonComparable(1, 0)
val y = NonComparable(2, 0)
//println(x < y)
//x.compareTo(y)

data class ComparableData(val value: Int): Comparable<ComparableData> {
    override fun compareTo(other: ComparableData): Int {
        return value.compareTo(other.value)
    }
}

val x2 = ComparableData(1)
val y2 = ComparableData(2)
println(x2 < y2)
println(x2.compareTo(y2))

true
-1


## 코틀린에서 compareTo를 따로 정의해야할까?
- 코틀린에서는 sortedBy 등의 메서드를 사용할 때, Comparable 인터페이스를 구현하지 않고 람다로 비교함수를 넘길 수 있다.


In [16]:
data class NonComparable(val value: Int, val value2: Int)

val list = listOf(NonComparable(3, 5), NonComparable(2, 3), NonComparable(1, 2), NonComparable(3, 3))

println(list.sortedBy { it.value }) // 단일 기준이든
println(list.sortedWith(compareBy({it.value}, {it.value2}))) // 다중 기준이든

[NonComparable(value=1, value2=2), NonComparable(value=2, value2=3), NonComparable(value=3, value2=5), NonComparable(value=3, value2=3)]
[NonComparable(value=1, value2=2), NonComparable(value=2, value2=3), NonComparable(value=3, value2=3), NonComparable(value=3, value2=5)]


- 만약 자연스럽지 않은 특정 비즈니스 기준을사용한다면 Comparator를 사용하는 것이 좋다.

In [20]:

data class User(val name: String, val age: Int) {
    companion object {
        val NameComparator = Comparator<User> { u1, u2 -> u1.name.compareTo(u2.name) }
        val AgeComparator = Comparator<User> { u1, u2 -> u1.age.compareTo(u2.age) }
    }
}

val users = listOf(User("Bob", 29), User("Alice", 31))

println(users.sortedWith(User.NameComparator))
println(users.sortedWith(User.AgeComparator))

[User(name=Alice, age=31), User(name=Bob, age=29)]
[User(name=Bob, age=29), User(name=Alice, age=31)]


## 그럼에도 compareTo를 구현해야 하는 경우
- 단일 프로퍼티 비교시엔 compareValues 함수를 사용해보자
- 다중 프로퍼티 비교시엔 compareValuesBy 함수를 사용해보자
- 정말 직접 구현한다면 다음을 기억하자
  - 리시버와 other가 같은경우 0을 반환해야 한다.
  - 리시버가 other보다 작은 경우 음수를 반환해야 한다.
  - 리시버가 other보다 큰 경우 양수를 반환해야 한다. 

In [ ]:
class Person(val firstName: String, val lastName: String): Comparable<Person> {
    override fun compareTo(other: Person): Int {
    return compareValues(this.lastName, other.lastName)
//        return compareValuesBy(this, other, Person::lastName, Person::firstName)
    }
}